In [1]:
import numpy as np
import pandas as pd

In [2]:
#importing files for processing
metadata=pd.read_csv('movies_metadata.csv')
links=pd.read_csv('links.csv')
ratings=pd.read_csv('ratings.csv')

C:\Users\gusai\AppData\Local\Temp\ipykernel_19084\1098401355.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata=pd.read_csv('movies_metadata.csv')


In [3]:
metadata.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [4]:
metadata.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [5]:
#these columns will used for linking index with title discarding non valuable data
metadata=metadata[['id','imdb_id','original_title']]

In [6]:
# converting data to numeric discarding values which are not convertable
metadata['id'] = pd.to_numeric(metadata['id'], errors='coerce') 

In [7]:
# Replacing NaN with 0 and converting to int
metadata['id'] = metadata['id'].fillna(0.0).astype(int) 

In [8]:
metadta=metadata.rename(columns={'id':'movieId'},inplace=True)

In [9]:
metadata.head()

,movieId,imdb_id,original_title
0,862,tt0114709,Toy Story
1,8844,tt0113497,Jumanji
2,15602,tt0113228,Grumpier Old Men
3,31357,tt0114885,Waiting to Exhale
4,11862,tt0113041,Father of the Bride Part II


In [10]:
links

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
45838,176269,6209470,439050.0
45839,176271,2028550,111109.0
45840,176273,303758,67758.0
45841,176275,8536,227506.0


In [11]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [12]:
#removing all the users vote whoose ratings count is less than 200 so that only experienced voter is chosen for recommendation
x=ratings['userId'].value_counts()>200
y=x[x].index
ratings=ratings[ratings['userId'].isin(y)]

In [13]:
ratings.shape

(15709678, 4)

In [14]:
#combining tables on movieId
rating_with_movies=ratings.merge(links,on='movieId')

In [15]:
rating_with_movies

,userId,movieId,rating,timestamp,imdbId,tmdbId
0,11,32,3.5,1231677006,114746,63.0
1,12,32,2.0,943911150,114746,63.0
2,24,32,4.0,979974219,114746,63.0
3,34,32,5.0,1498760931,114746,63.0
4,46,32,3.0,979331727,114746,63.0
...,...,...,...,...,...,...
15709673,270887,162358,2.0,1473989924,3616894,407455.0
15709674,270887,165053,4.0,1491183181,1949605,415766.0
15709675,270887,165649,4.0,1479088587,3731196,302042.0
15709676,270887,171051,4.0,1493084042,176326,23476.0


In [16]:
# Replacing NaN with 0 and converting to int
rating_with_movies['tmdbId'] =rating_with_movies['tmdbId'].fillna(0.0).astype(int)

In [17]:
rating_with_movies=rating_with_movies[['userId','rating','tmdbId','imdbId']]

In [18]:
metadata

,movieId,imdb_id,original_title
0,862,tt0114709,Toy Story
1,8844,tt0113497,Jumanji
2,15602,tt0113228,Grumpier Old Men
3,31357,tt0114885,Waiting to Exhale
4,11862,tt0113041,Father of the Bride Part II
...,...,...,...
45461,439050,tt6209470,رگ خواب
45462,111109,tt2028550,Siglo ng Pagluluwal
45463,67758,tt0303758,Betrayal
45464,227506,tt0008536,Satana likuyushchiy


In [19]:
#removing extra 'tt' from the column values of imdbId
metadata["imdb_id"]=metadata["imdb_id"].str.replace("tt","")

In [20]:
metadata

,movieId,imdb_id,original_title
0,862,0114709,Toy Story
1,8844,0113497,Jumanji
2,15602,0113228,Grumpier Old Men
3,31357,0114885,Waiting to Exhale
4,11862,0113041,Father of the Bride Part II
...,...,...,...
45461,439050,6209470,رگ خواب
45462,111109,2028550,Siglo ng Pagluluwal
45463,67758,0303758,Betrayal
45464,227506,0008536,Satana likuyushchiy


In [21]:
# Replacing NaN with 0 and converting imdb_id coloumn from string to integer 
metadata['imdb_id'] = metadata['imdb_id'].fillna(0.0).astype(int)

In [22]:
rating_with_movies

,userId,rating,tmdbId,imdbId
0,11,3.5,63,114746
1,12,2.0,63,114746
2,24,4.0,63,114746
3,34,5.0,63,114746
4,46,3.0,63,114746
...,...,...,...,...
15709673,270887,2.0,407455,3616894
15709674,270887,4.0,415766,1949605
15709675,270887,4.0,302042,3731196
15709676,270887,4.0,23476,176326


In [23]:
#renaming to imdbId so that it can later be merged 
metadata.rename(columns={'imdb_id':'imdbId'},inplace=True)

In [24]:
metadata

,movieId,imdbId,original_title
0,862,114709,Toy Story
1,8844,113497,Jumanji
2,15602,113228,Grumpier Old Men
3,31357,114885,Waiting to Exhale
4,11862,113041,Father of the Bride Part II
...,...,...,...
45461,439050,6209470,رگ خواب
45462,111109,2028550,Siglo ng Pagluluwal
45463,67758,303758,Betrayal
45464,227506,8536,Satana likuyushchiy


In [25]:
metadata=metadata.merge(links,on='imdbId')

In [26]:
metadata

,movieId_x,imdbId,original_title,movieId_y,tmdbId
0,862,114709,Toy Story,1,862.0
1,8844,113497,Jumanji,2,8844.0
2,15602,113228,Grumpier Old Men,3,15602.0
3,31357,114885,Waiting to Exhale,4,31357.0
4,11862,113041,Father of the Bride Part II,5,11862.0
...,...,...,...,...,...
45378,439050,6209470,رگ خواب,176269,439050.0
45379,111109,2028550,Siglo ng Pagluluwal,176271,111109.0
45380,67758,303758,Betrayal,176273,67758.0
45381,227506,8536,Satana likuyushchiy,176275,227506.0


In [27]:
#excluding unwanted data
metadata=metadata[['tmdbId','original_title','imdbId']]

In [28]:
rating_with_movies=rating_with_movies.merge(metadata,on='imdbId')

In [29]:
rating_with_movies

,userId,rating,tmdbId_x,imdbId,tmdbId_y,original_title
0,11,3.5,63,114746,63.0,Twelve Monkeys
1,12,2.0,63,114746,63.0,Twelve Monkeys
2,24,4.0,63,114746,63.0,Twelve Monkeys
3,34,5.0,63,114746,63.0,Twelve Monkeys
4,46,3.0,63,114746,63.0,Twelve Monkeys
...,...,...,...,...,...,...
15675456,270887,2.0,407455,3616894,407455.0,The Chemist
15675457,270887,4.0,415766,1949605,415766.0,Who Gets the Dog?
15675458,270887,4.0,302042,3731196,302042.0,Dead on Campus
15675459,270887,4.0,23476,176326,23476.0,Without Evidence


In [30]:
#excluding unwanted data
rating_with_movies=rating_with_movies[['userId','rating','tmdbId_x','original_title']]

In [31]:
#renaming to tmdbId so that it can later be merged 
rating_with_movies.rename(columns={'tmdbId_x':'tmdbId'},inplace=True)

C:\Users\gusai\AppData\Local\Temp\ipykernel_19084\3739241873.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating_with_movies.rename(columns={'tmdbId_x':'tmdbId'},inplace=True)


In [32]:
rating_with_movies

,userId,rating,tmdbId,original_title
0,11,3.5,63,Twelve Monkeys
1,12,2.0,63,Twelve Monkeys
2,24,4.0,63,Twelve Monkeys
3,34,5.0,63,Twelve Monkeys
4,46,3.0,63,Twelve Monkeys
...,...,...,...,...
15675456,270887,2.0,407455,The Chemist
15675457,270887,4.0,415766,Who Gets the Dog?
15675458,270887,4.0,302042,Dead on Campus
15675459,270887,4.0,23476,Without Evidence


In [33]:
#counting no. of ratings per movie 
number_rating=rating_with_movies.groupby('tmdbId')['rating'].count().reset_index()

In [34]:
number_rating.rename(columns={'rating':'no_of_ratings'},inplace=True)

In [35]:
number_rating

,tmdbId,no_of_ratings
0,2,223
1,3,82
2,5,3207
3,6,1061
4,11,25611
...,...,...
43912,464819,1
43913,467731,1
43914,468343,4
43915,468707,1


In [36]:
#combining it again so that we have value for each
final_rating=rating_with_movies.merge(number_rating,on ='tmdbId')

In [37]:
final_rating

,userId,rating,tmdbId,original_title,no_of_ratings
0,11,3.5,63,Twelve Monkeys,19984
1,12,2.0,63,Twelve Monkeys,19984
2,24,4.0,63,Twelve Monkeys,19984
3,34,5.0,63,Twelve Monkeys,19984
4,46,3.0,63,Twelve Monkeys,19984
...,...,...,...,...,...
15675456,270887,2.0,407455,The Chemist,1
15675457,270887,4.0,415766,Who Gets the Dog?,1
15675458,270887,4.0,302042,Dead on Campus,1
15675459,270887,4.0,23476,Without Evidence,1


In [38]:
#selecting only those movies which have more than 200 ratings on them
final_rating=final_rating[final_rating['no_of_ratings']>=200] 

In [39]:
final_rating.shape

(14924056, 5)

In [40]:
final_rating.drop_duplicates(['userId','tmdbId'], inplace=True)

C:\Users\gusai\AppData\Local\Temp\ipykernel_19084\330106811.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_rating.drop_duplicates(['userId','tmdbId'], inplace=True)


In [41]:
final_rating.shape

(14922494, 5)

In [42]:
#creating a table having rows as tmdbId and coloumns as userId containing rating as values
movie_pivot=final_rating.pivot_table(columns='userId', index='tmdbId',values='rating')

In [43]:
movie_pivot

userId,11,12,15,20,24,34,43,46,47,49,...,270807,270828,270850,270857,270859,270871,270872,270879,270887,270896
tmdbId,,,,,,,,,,,,,,,,,,,,,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,3.0,5.0,1.5,4.0,5.0,5.0,3.5,4.5,NaN,5.0,...,4.0,5.0,5.0,4.0,NaN,5.0,4.5,5.0,5.0,NaN
12,3.5,NaN,4.0,5.0,NaN,1.5,3.5,4.0,NaN,NaN,...,4.0,NaN,NaN,3.5,NaN,NaN,NaN,NaN,1.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396535,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
399106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
movie_pivot.fillna(0,inplace=True)# replacing NaN with 0

In [45]:
movie_pivot

userId,11,12,15,20,24,34,43,46,47,49,...,270807,270828,270850,270857,270859,270871,270872,270879,270887,270896
tmdbId,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,3.0,5.0,1.5,4.0,5.0,5.0,3.5,4.5,0.0,5.0,...,4.0,5.0,5.0,4.0,0.0,5.0,4.5,5.0,5.0,0.0
12,3.5,0.0,4.0,5.0,0.0,1.5,3.5,4.0,0.0,0.0,...,4.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,1.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396535,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
399106,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
410199,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
from scipy.sparse import csr_matrix
#making matrix out of table
movie_sparse=csr_matrix(movie_pivot)

In [47]:
type(movie_sparse)

scipy.sparse._csr.csr_matrix

In [48]:
from sklearn.neighbors import NearestNeighbors
#selecting Nearest neighbors for modelling
model=NearestNeighbors(algorithm='brute') 

In [49]:
model.fit(movie_sparse)

NearestNeighbors(algorithm='brute')

In [50]:
#this gives distances of neighbor and their index
distances,suggestion= model.kneighbors(movie_pivot.iloc[237,:].values.reshape(1,-1),n_neighbors=21)

In [51]:
suggestion #index of recommendations

array([[ 237, 6375, 5605, 5493, 4626, 5737, 6057, 2662, 1523, 5541, 3852,
        5968, 6065, 2306, 6525, 4731, 3718, 2927, 5975, 5321, 3129]],
      dtype=int64)

In [52]:
for i in suggestion:
    print(movie_pivot.index[i])

Int64Index([  336, 58048, 31246, 28605, 17644, 33875, 41671, 10253,  5227,
            29794, 12779, 39128, 41848,  9624, 74997, 18423, 12236, 10696,
            39283, 26263, 11059],
           dtype='int64', name='tmdbId')


In [53]:
metadata['original_title']=metadata['original_title'].astype(str)

C:\Users\gusai\AppData\Local\Temp\ipykernel_19084\34419019.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata['original_title']=metadata['original_title'].astype(str)


In [54]:
movies=metadata.sort_values(by = 'tmdbId').reset_index() #making a new table with row sorted according to 'tmbdId'

In [55]:
movies=movies[['tmdbId','original_title']]

In [56]:
movies=movies.merge(final_rating,on='tmdbId') #merging file on the basis of tmbdId with final rating 

In [57]:
movies.drop_duplicates('tmdbId', inplace=True)

In [58]:
movies=movies[['tmdbId','original_title_x']]


In [59]:
movies.rename(columns={'original_title_x':'title'},inplace=True)

In [60]:
movies=movies.reset_index()

In [61]:
movies=movies[['tmdbId','title']]

In [62]:
movies[movies['title']=='Sully' ].index[0]

7177

In [63]:
movies['tmdbId'] = movies['tmdbId'].astype(int)

In [64]:
movies['tmdbId'].iloc[7177]

363676

In [66]:
def recommend1(movie):
    #A function which accept movie name and returns the index of recommendations in the movies dataframe
    index =movies[movies['title']==movie].index[0]
    distances,suggestion= model.kneighbors(movie_pivot.iloc[index,:].values.reshape(1,-1),n_neighbors=21)
    L=[]
    for i in suggestion[0][1:]:
        L.append(i)
    return L

In [67]:
movies['recommendations']=movies['title'].apply(recommend1) #applying recommend1 to all the rows of movies

In [72]:
movies.rename(columns={'tmdbId':'id'},inplace=True)

In [73]:
import pickle
pickle.dump(movies,open('userbased.pkl','wb')) #dumping the final processed file